## OpenAI agents Playground

# 🤖 OpenAI Agents Project

## 🎯 Long-term Goals
- 📚 Learn and understand OpenAI's agent architecture and capabilities
- 🔧 Create custom agents for specific use cases and domains
- 🔄 Integrate agents into other projects and workflows
- 🧩 Experiment with multi-agent systems and orchestration

## 🚀 Features
- Agent-based architecture with specialized roles
- Handoff system for intelligent task routing
- Guardrails for safe and controlled agent behavior
- Async execution for efficient processing

## 💡 Use Cases
- Educational assistants and tutoring
- Research and information gathering
- Workflow automation
- Content generation and curation

## 🛠️ Technologies Used
- OpenAI's agent framework
- Python async programming
- Pydantic for data validation
- Jupyter for interactive development

#### install

In [19]:
# ! pip install openai-agents nest_asyncio


### imports

In [13]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput
import asyncio
from IPython.display import display, Markdown

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [2]:
nest_asyncio.apply()

#### Load environment variables

In [3]:
dotenv.load_dotenv()

True

#### First Agent - Math Agent

In [4]:
math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

#### Second and Third Agents - History and Science Agents

In [5]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

science_tutor_agent = Agent(
    name="Science Tutor",
    handoff_description="Specialist agent for science questions",
    instructions="You provide help with science problems. Explain concepts clearly and provide examples.",
)

### Handoffs definition

In [6]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, science_tutor_agent, math_tutor_agent]
)

### Agents Orchestration

In [17]:
async def ask_ai(q: str) -> GuardrailFunctionOutput:
    result = await Runner.run(triage_agent, q)
    display(Markdown(result.final_output))

### Adding Guardrails

In [8]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

###### question

In [9]:
question: str = "What is the Pythagorean theorem?"

#### run main()

In [18]:
if __name__ == "__main__":
    asyncio.run(ask_ai(question))

The Pythagorean Theorem is a fundamental principle in geometry that applies to right-angled triangles. It states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.

The theorem is usually written as:

\[ a^2 + b^2 = c^2 \]

where:
- \( c \) is the length of the hypotenuse,
- \( a \) and \( b \) are the lengths of the other two sides.

### Example:

Consider a right-angled triangle where one side is 3 units long, the other side is 4 units long, and you need to find the hypotenuse.

1. **Identify the sides:** Let's say \( a = 3 \) and \( b = 4 \).

2. **Apply the Pythagorean Theorem:**
   \[
   a^2 + b^2 = c^2
   \]

3. **Plug in the values:**
   \[
   3^2 + 4^2 = c^2 \\
   9 + 16 = c^2 \\
   25 = c^2
   \]

4. **Solve for \( c \) (the hypotenuse):**
   \[
   c = \sqrt{25} \\
   c = 5
   \]

Thus, the hypotenuse of the triangle is 5 units.

This theorem is widely used in various fields, such as construction, navigation, and physics, whenever dealing with right-angled triangles.